In [11]:
import pandas as pd
from glob import glob
import itertools
from collections import Counter
import numpy as np
import numpy
import math
import matplotlib.pylab as plt
import os
import networkx as nx
from pypinyin import lazy_pinyin

In [67]:
# first put all the unique nodes together, second all pair combinations together,third pair combinations for every user
files=glob('./siteexample1/allusers/*')
for f in files:
    user=f.split('/')[-1].split('.')[0]
    userdata=pd.read_csv(f,names=['user_id','longtitude','latitude','app_name','app_type_name','start_time'])
    
    userdata=userdata[userdata['app_name']!='其他']
    userdata=userdata[['user_id','app_name']]
    userdata=userdata.drop_duplicates()
    
    userdata.to_csv('./siteexample1/node.csv',header=False,mode='a+')
    
    app_list=userdata['app_name'].values
    pair=list(itertools.combinations(app_list,2))
    data=pd.DataFrame(pair,columns=['node1','node2'])

    data['node3']=data['node1'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    data['node4']=data['node2'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    for i in range(len(data.values)):
        if data.values[i][2]>data.values[i][3]:
            data.values[i][0],data.values[i][1]=data.values[i][1],data.values[i][0]
            
    data=data[['node1','node2']]
    

    data.to_csv('./siteexample1/pair.csv',header=False,mode='a+')
    
    with open('./siteexample1/pair/' + str(user) +'.csv', 'a+') as f:
        data.to_csv(f,header=False)

In [13]:
# observed values
app_obs=pd.read_csv('./siteexample1/pair.csv',names=['node1','node2'],index_col=0)

# app_obs['node3']=app_obs['node1'].map(lambda x: ''.join(lazy_pinyin(str(x))))
# app_obs['node4']=app_obs['node2'].map(lambda x: ''.join(lazy_pinyin(str(x))))
# for i in range(len(app_obs.values)):
#     if app_obs.values[i][2]>app_obs.values[i][3]:
#         app_obs.values[i][0],app_obs.values[i][1]=app_obs.values[i][1],app_obs.values[i][0]

app_obs['pair']=app_obs['node1'].astype('str')+' '+app_obs['node2'].astype('str')
counterdict=Counter(app_obs['pair'].values)
counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
counterdict.to_csv('./siteexample1/observed_value.csv')    

In [14]:
# expected values
for i in range (10): # reconstruct random network for 10 times
    name='app_exp'+str(i)
    name=pd.read_csv('./siteexample1/node.csv',index_col=0,names=['user_id','app_name'])
    numpy.random.shuffle(name['app_name'].values) # shuffle the data
        
    users=name['user_id'].drop_duplicates()
    groups=name.groupby('user_id')
    
    path='./siteexample1/expected_value/'
    f=path+'app_exp'+str(i)+'.csv'

    for user in users:
        userdata=groups.get_group(user)
        userdata=userdata.drop_duplicates()

        app_list=userdata['app_name'].values
        pair=list(itertools.combinations(app_list,2))
        app_exp=pd.DataFrame(pair)
        app_exp.to_csv(f,header=0,mode='a+')

    app_exp=pd.read_csv(f,names=['node1','node2'])
    app_exp['node3']=app_exp['node1'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    app_exp['node4']=app_exp['node2'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    for j in range(len(app_exp.values)):
        if app_exp.values[j][2]>app_exp.values[j][3]:
            app_exp.values[j][0],app_exp.values[j][1]=app_exp.values[j][1],app_exp.values[j][0]
    app_exp['pair']=app_exp['node1'].astype('str')+' '+app_exp['node2'].astype('str')
    counterdict=Counter(app_exp['pair'].values)
    counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
    counterdict.to_csv(f,header=0,mode='w')

In [15]:
# combine observed value and expected value
files=glob('./siteexample1/expected_value/*')

for f in files:
    name=f.split('/')[-1].split('.')[0]
    data=pd.read_csv(f,names=['pair','count'])
    diction=dict(data[['pair','count']].values)
    z_score=pd.read_csv('./siteexample1/observed_value.csv',index_col=0)
    z_score[name]=z_score['pair'].apply(lambda x: diction[x]
                                                    if x in diction
                                                    else numpy.nan)
    z_score.to_csv('./siteexample1/observed_value.csv')

In [37]:
# z_score
z_score=pd.read_csv('./siteexample1/observed_value.csv',index_col=0)
z_score=z_score.fillna(0)
mean=z_score.iloc[:,[x for x in range(2,12)]].mean(axis=1)
std=z_score.iloc[:,[x for x in range(2,12)]].std(axis=1)
z_score['z_score']=(z_score['app_obs']-mean)/std
z_score.to_csv('./siteexample1/observed_value.csv')

In [62]:
z_score=z_score[~z_score.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]
# z_score[z_score['z_score']==np.inf]

,pair,app_obs,app_exp7,app_exp6,app_exp4,app_exp5,app_exp1,app_exp0,app_exp2,app_exp3,app_exp8,app_exp9,z_score


In [68]:
# put z_score into every user-file
z_score=pd.read_csv('./siteexample1/observed_value.csv',usecols=['pair','z_score'])
diction=dict(z_score.values)

files=glob('./siteexample1/pair/*')

for f in files:
    data=pd.read_csv(f,names=['node1','node2'])
    data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
    data['z_score']=data['pair'].apply(lambda x: diction[x]
                                  if x in diction
                                   else np.nan)
    data.to_csv(f)

In [69]:
# four categories group by z_score
files=glob('./siteexample1/pair/*')
lista=[]
listb=[]
listc=[]
for f in files:
    data=pd.read_csv(f)
    if len(data['z_score'])>0:
        user_id=f.split('/')[-1].split('.')[0]
        lista.append(user_id)
        a=numpy.array(data['z_score'].values)
        median=numpy.median(a)
        listb.append(median)
        pertentile_10=numpy.percentile(a,10)
        listc.append(pertentile_10)
df=pd.DataFrame([lista,listb,listc]).T
df.columns=['user_id','median','pertentile_10']
df.to_csv('./siteexample1/zscore_category.csv')

In [70]:
# add category

df=pd.read_csv('./siteexample1/zscore_category.csv',index_col=0)
df=df[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

high_low=df[(df['median']>df['median'].median())&(df['pertentile_10']>0)].index
df.ix[high_low,'category']='high_low'


low_low=df[(df['median']<=df['median'].median())&(df['pertentile_10']>0)].index
df.ix[low_low,'category']='low_low'

low_high=df[(df['median']<=df['median'].median())&(df['pertentile_10']<=0)].index
df.ix[low_high,'category']='low_high'

high_high=df[(df['median']>df['median'].median())&(df['pertentile_10']<=0)].index
df.ix[high_high,'category']='high_high'

df=df[['user_id','median','pertentile_10','category']]
df.to_csv('./siteexample1/zscore_category.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [71]:
columns = 'user_id，sex，age，occupation_name，education_name，user_opentime，brand_name，call_duration_m，gprs_flow，\
call_fee，gprs_fee，databusiness_fee，order_name，brand_chn，model_chn，screensize，operation_sys，terminal_price，\
dept_county_name，dept_name'.split('，')

basename=pd.read_csv('./basename.csv', encoding='gbk', names=columns)

basename_except=basename.drop(['order_name'],axis=1)
basename_except=basename_except.drop_duplicates()

basename_except['sex']=basename_except['sex'].replace('不详',np.nan)

basename_except['age']=basename_except['age'].replace(0,np.nan)

basename_except['terminal_price']=basename_except['terminal_price'].replace(0,np.nan)

basename_except['fees']=basename_except['call_fee']+basename_except['gprs_fee']+basename_except['databusiness_fee']

basename_except=basename_except[['user_id','sex','age','fees','brand_chn','terminal_price']]

basename_except=basename_except.dropna(axis=0,how='any')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
basename_except['user_id'].nunique()

4125

In [73]:
# fetch the valid information sex age fees brand terminal_price to zscore_category

df=pd.read_csv('./siteexample1/zscore_category.csv',index_col=0)

names=['sex','age','fees','brand_chn','terminal_price']
for n in names:
    diction=dict(basename_except[['user_id',n]].values)
    df[n]=df['user_id'].apply(lambda x: diction[x]
                                 if x in diction
                                 else numpy.nan)


df.to_csv('./siteexample1/zscore_category.csv')

In [75]:
zscore_category=pd.read_csv('./siteexample1/zscore_category.csv',index_col=0)

In [76]:
zscore_category['user_id'].nunique()

3990

In [90]:
zscore_category.sort_values(by=['median'],ascending=False)[:int(len(zscore_category)*0.05)]

,user_id,median,pertentile_10,category,sex,age,fees,brand_chn,terminal_price,category1
1055,35432566,90.967914,69.161939,high_low,男,27.0,52.20,诺基亚,989.0,convention
935,30332999,90.967914,68.495602,high_low,男,36.0,72.77,苹果,4199.0,convention
374,2708704,90.967914,90.967914,high_low,男,32.0,62.38,小米,1649.0,convention
82,31787801,90.967914,68.495602,high_low,男,44.0,61.50,苹果,3188.0,convention
2179,41520363,90.967914,68.495602,high_low,女,24.0,59.24,三星,2159.0,convention
2390,31361317,90.967914,68.495602,high_low,女,34.0,156.13,苹果,2599.0,convention
3020,40387120,90.967914,68.495602,high_low,男,31.0,29.90,三星,799.0,convention
516,36359907,90.967914,68.495602,high_low,女,32.0,98.98,诺基亚,699.0,convention
1564,57759331,90.967914,68.495602,high_low,男,28.0,63.89,苹果,4199.0,convention
963,57354939,90.967914,68.495602,high_low,男,30.0,113.10,黑莓,1999.0,convention
